# Population by Fed District

This file contains code to obtain data by Federal Reserve district for:
1. Population (annual since 1970)
2. Unemployment (monthly, 1990-2015)
3. Job levels (monthly, 1990-present)

In [1]:
# Setting up FRED access
import pandas as pd
from fredapi import Fred
api_key = "b40f40a14d67c7903b44700db6b7b6e7"
fred = Fred(api_key=api_key)

In [2]:
# Giving the FRED series IDs for population for each of the 12 Fed districts
frb_series_ids_pop = [
    "D1POP", "D2POP", "D3POP", "D4POP", "D5POP", "D6POP",
    "D7POP", "D8POP", "D9POP", "D10POP", "D11POP", "D12POP"
    ]
frb_ids_pop = pd.DataFrame([frb_series_ids_pop])

# Blank dataframe setup
frb_pop = pd.DataFrame()

In [3]:
# Loop to download data for each state
for series_id in frb_series_ids_pop:
    frb_data_pop = fred.get_series(series_id)
    frb_pop = pd.concat([frb_pop, frb_data_pop], axis = 1)

In [30]:
# Rename columns by state
frb_pop.columns = frb_series_ids_pop
frb_pop.tail()

#Export to CSV
#frb_pop.to_csv("frb_pop.csv")

,D1POP,D2POP,D3POP,D4POP,D5POP,D6POP,D7POP,D8POP,D9POP,D10POP,D11POP,D12POP
2019-01-01 00:00:00,13905.274,26511.491,13323.775,17049.976,32627.504,47838.758,34777.256,14791.437,9524.669,18544.311,30629.306,68758.891
2020-01-01 00:00:00,14115.143,27419.765,13596.615,17212.970,32712.201,48256.354,35126.596,14787.291,9657.192,18652.648,30883.711,69059.868
2021-01-01 00:00:00,14156.098,27160.760,13650.138,17166.181,32902.547,48595.341,35015.826,14795.556,9689.177,18716.324,31199.904,68954.917
2022-01-01 00:00:00,14170.757,26962.749,13654.551,17137.097,33123.822,49211.078,34935.650,14799.527,9724.822,18779.955,31657.411,69068.300
2023-01-01 00:00:00,14208.219,26880.474,13682.232,17153.833,33412.114,49800.601,34962.488,14830.385,9777.696,18879.442,32126.022,69155.967


In [25]:
# Calculate change in population from the previous year
frb_pop_change = frb_pop.pct_change()

# Rename columns as D1POP_change, D2POP_change, etc.
frb_pop_change.columns = [f"{col}_change" for col in frb_pop_change.columns]

# Merge the original population data with the change in population data
frb_pop_change = pd.concat([frb_pop, frb_pop_change], axis = 1)

# Export as CSV
#frb_pop_change.to_csv("Fed_District_population.csv")

In [ ]:
# This is all for adding the population data to the BB data, which is not as easy as it seems!
    # See the file Beige Book classification models.ypynb for where I read this in
    # I'm just storing here so it won't jumble up that file

# read in bbNoText.csv as bbSimple
bbSimple = pd.read_csv("bbNoText.csv")

# Replace Nan with 0
bbSimple.fillna(0, inplace=True)

# Change Date to datetime format
bbSimple['Date'] = pd.to_datetime(bbSimple['Date'])

# Read in population data by Fed district
pop = pd.read_csv("frb_pop.csv")

# Set Date as datetime
pop['Date'] = pd.to_datetime(pop['Date'])

# Add rows for every month in the data starting on the first of the month
# Include the same population number for each month within a year
pop = pop.set_index('Date').resample('MS').ffill().reset_index()

# Assuming 2024 is same as 2023 because we don't have data for 2024
# Drop 2025 data, which was just included to make it fill in all months for 2024
pop = pop[pop['Date'].dt.year < 2025]

# Join bbSimple and pop while keeping all data in bbSimple
bbSimple = pd.merge(bbSimple, pop, on=['Date'], how='left')

# Sum the population for each district into a new column called D13POP
bbSimple['D13POP'] = bbSimple['D01POP'] + bbSimple['D02POP'] + bbSimple['D03POP'] + bbSimple['D04POP'] + bbSimple['D05POP'] + bbSimple['D06POP'] + bbSimple['D07POP'] + bbSimple['D08POP'] + bbSimple['D09POP'] + bbSimple['D10POP'] + bbSimple['D11POP'] + bbSimple['D12POP']

# For DO1POP to D12POP, want to place the values into a new column called 'Population' based on the District number
# Create a dictionary to map the District number to the column name
pop_dict = {'D01POP': '1', 'D02POP': '2', 'D03POP': '3', 'D04POP': '4', 'D05POP': '5', 'D06POP': '6', 'D07POP': '7', 'D08POP': '8', 'D09POP': '9', 'D10POP': '10', 'D11POP': '11', 'D12POP': '12', 'D13POP': '13'}

# Loop through the dictionary and place the values into the 'Population' column
for key, value in pop_dict.items():
    bbSimple.loc[bbSimple['District'] == int(value), 'population'] = bbSimple[key]

# Drop the columns that are no longer needed
bbSimple.drop(columns=['D01POP', 'D02POP', 'D03POP', 'D04POP', 'D05POP', 'D06POP', 'D07POP', 'D08POP', 'D09POP', 'D10POP', 'D11POP', 'D12POP','D13POP'], inplace=True)

# Export to csv
#bbSimple.to_csv("bbNoText_pop.csv")

In [ ]:
# This calculates percent change in population from the previous year for each district
    # Not using it for now but leaving it here in case want to use later
# For each month, look up the year, then calculate the percent change in population from the previous year for that distict
bbSimple['year'] = bbSimple['Date'].dt.year
bbSimple['prev_year'] = bbSimple['year'] - 1

# Create a new column for the previous year's population
bbSimple['prev_year_pop'] = bbSimple['population'].shift(12)

# Calculate the percent change in population from the previous year
bbSimple['pop_change'] = (bbSimple['population'] - bbSimple['prev_year_pop']) / bbSimple['prev_year_pop']

# Drop the columns that are no longer needed
bbSimple.drop(columns=['year', 'prev_year', 'prev_year_pop'], inplace=True)


# Unemployment Rate by Fed District

Dataset available 1990 - 2015

In [5]:
# Giving the FRED series IDs for unemployment rate each of the 50 states
    # Doing this in case I decide to use unemployment rate as a DV
frb_series_ids = [
    "D1URN", "D2URN", "D3URN", "D4URN", "D5URN", "D6URN",
    "D7URN", "D8URN", "D9URN", "D10URN", "D11URN", "D12URN"
    ]
frb_ids = pd.DataFrame([frb_series_ids])

In [6]:
frb_ur = pd.DataFrame()

# Loop to download data for each Fed district
for series_id in frb_series_ids:
    frb_data = fred.get_series(series_id)
    frb_ur = pd.concat([frb_ur, frb_data], axis = 1)

In [7]:
# Rename columns by district
frb_ur.columns = frb_series_ids
frb_ur.head()

,D1URN,D2URN,D3URN,D4URN,D5URN,D6URN,D7URN,D8URN,D9URN,D10URN,D11URN,D12URN
1990-01-01 00:00:00,5.682,5.544,5.612,6.772,4.707,6.002,6.784,6.973,5.713,5.671,6.636,5.724
1990-02-01 00:00:00,5.745,5.555,5.599,6.301,4.573,5.867,6.515,7.126,5.709,5.316,6.625,5.668
1990-03-01 00:00:00,5.654,5.172,5.231,5.694,4.071,5.473,6.202,6.545,5.673,5.037,6.161,5.416
1990-04-01 00:00:00,5.440,4.878,4.892,5.796,4.028,5.554,5.756,6.154,5.093,4.656,6.137,5.276
1990-05-01 00:00:00,5.442,4.851,4.955,5.460,4.237,5.569,5.509,5.951,4.372,4.622,6.167,5.052


In [8]:
frb_ur.tail()
# This data is not seasonally adjusted
# Federal Reserve Bank of Dallas has employment levels data going back to 1990 that is still updated

,D1URN,D2URN,D3URN,D4URN,D5URN,D6URN,D7URN,D8URN,D9URN,D10URN,D11URN,D12URN
2015-07-01 00:00:00,4.873,5.546,6.008,5.288,5.742,6.102,5.338,5.987,3.814,4.612,4.718,6.153
2015-08-01 00:00:00,4.547,5.078,5.579,4.642,5.478,5.779,4.842,5.262,3.490,4.346,4.516,5.857
2015-09-01 00:00:00,4.389,4.907,5.063,4.521,5.096,5.563,4.415,4.957,3.203,3.884,4.492,5.371
2015-10-01 00:00:00,4.279,4.599,4.797,4.338,5.120,5.384,4.596,4.833,3.055,3.844,4.607,5.446
2015-11-01 00:00:00,4.384,4.681,4.629,4.551,4.985,5.118,4.674,4.918,3.181,3.821,4.648,5.440


# Jobs by Fed district

In [9]:
# Dallas Fed posts updated Excel file each month at this length
    # Data goes back to 1990
read_url = "https://www.dallasfed.org/~/media/documents/research/econdata//emp12dist/emp12dist.xlsx"

In [10]:
# Read in data from Excel file

emp12dist = pd.read_excel(read_url, sheet_name = "12distsa", skiprows = 3, index_col = 0)

In [11]:
# Going to rename columns to match other data by District
new_column_names = {col: f'D{i}_jobs' for i, col in enumerate(emp12dist.columns, start=1)}

emp12dist.rename(columns=new_column_names, inplace=True)
emp12dist.head()


,D1_jobs,D2_jobs,D3_jobs,D4_jobs,D5_jobs,D6_jobs,D7_jobs,D8_jobs,D9_jobs,D10_jobs,D11_jobs,D12_jobs
Date,,,,,,,,,,,,
1990-01-01,6056.2,11350.1,5178.8,6875.4,10961.1,13350.9,13980.7,5207.5,3354.4,5794,7467.7,19726.3
1990-02-01,6043.3,11346.6,5187.1,6904.6,11002.3,13378.7,14046.5,5205.1,3361.2,5808,7478.9,19757.4
1990-03-01,6041.8,11349,5195,6924.3,11028.6,13411.5,14079.4,5212.3,3364,5820.9,7499.8,19790.9
1990-04-01,6001.5,11288,5164.8,6930.2,11010.5,13402,14061.3,5209.3,3370.4,5828.2,7522.5,19841.1
1990-05-01,5992.9,11301.6,5163.3,6943,11024.6,13440.3,14082,5220.2,3377.4,5853,7565,19883.6


Data available from FRED by counties:
1. Per capita personal income (since 1969) - PCPI is the code, followed by 5-digit GEOID
    Example: Autauga County, AL: https://fred.stlouisfed.org/series/PCPI01001, GEOID from FedCounties is 1001
        Might need to add that leading 0 to the counties that only have 4 digits

2. Percent below the poverty line (annual, since 1989) - S1701ACS followed by 6-digit GEOID
    Autauga, AL: https://fred.stlouisfed.org/series/S1701ACS001001

3. Unemployment rate (monthly, since 1990) 
    Autauga, Al: https://fred.stlouisfed.org/series/ALAUTA1URN
    Baldwin County, Al: https://fred.stlouisfed.org/series/ALBALD0URN

4. Civilian labor force (monthly, since 1990)
    Autauga, AL: https://fred.stlouisfed.org/series/ALAUTA1LFN

5. House Price Index (annual, since 1975)
    Big counties only, e.g. Mobile County - https://fred.stlouisfed.org/series/ATNHPIUS01097A\

6. SNAP benefits recipients (annual, since 1989)
    Autauga, AL: https://fred.stlouisfed.org/series/CBR01001ALA647NCEN



# County listing by Fed District

Source: Tousey, Colton. 2019. Data files for the paper "Federal Reserve District County Shapefiles." Federal Reserve Bank of Kansas City, Technical Briefing no. 19-01. Available at https://www.kansascityfed.org/publications/research/technicalbriefings/articles/2019/0417-district-county-shapefiles

In [13]:
import pandas as pd
FedCounties = pd.read_csv('FedCounties.csv', encoding='latin1')

In [14]:
# Add one leading zero to GEOID if it only has 4 characters and make it a string
FedCounties['GEOID'] = FedCounties['GEOID'].apply(lambda x: str(x).zfill(5))

# add "PCPI" to beginning of each GEOID
FedCounties['GEOID'] = FedCounties['GEOID'].apply(lambda x: "PCPI" + x)



In [10]:
# This takes a long time to loop through all the series IDs: 35 minutes in my case

frb_pcpi = pd.DataFrame()

for series_id in FedCounties['GEOID']:
    try:
        frb_data_pcpi = fred.get_series(series_id)
        frb_pcpi = pd.concat([frb_pcpi, frb_data_pcpi], axis=1)
    except Exception as e:
        print(f"Error occurred while processing series ID {series_id}: {e}")
        continue


Error occurred while processing series ID PCPI02158: Bad Request.  The series does not exist.
Error occurred while processing series ID PCPI15005: Bad Request.  The series does not exist.
Error occurred while processing series ID PCPI15009: Bad Request.  The series does not exist.
Error occurred while processing series ID PCPI51003: Bad Request.  The series does not exist.
Error occurred while processing series ID PCPI51005: Bad Request.  The series does not exist.
Error occurred while processing series ID PCPI51015: Bad Request.  The series does not exist.
Error occurred while processing series ID PCPI51031: Bad Request.  The series does not exist.
Error occurred while processing series ID PCPI51035: Bad Request.  The series does not exist.
Error occurred while processing series ID PCPI51053: Bad Request.  The series does not exist.
Error occurred while processing series ID PCPI51059: Bad Request.  The series does not exist.
Error occurred while processing series ID PCPI51069: Bad Req

In [15]:
# Read in list of 145 bad_pcpi_ids
    # These are the series IDs that returned an error when trying to download data
    # Good explanations for most of these errors:
    # Name change in Alaska, small counties in Hawaii, Virginia cities, in Guam and other U.S. territories, etc.
bad_pcpi_ids = pd.read_csv('bad_pcpi_ids.csv',  header= None)

In [16]:
# Remove bad series IDs from FedCounties
FedCounties2 = FedCounties[~FedCounties['GEOID'].isin(bad_pcpi_ids[0])]  

In [17]:
# Rename columns in frb_pcpi to match FedCounties2 GEOIDs
frb_pcpi.columns = FedCounties2['GEOID'][:len(frb_pcpi.columns)]


NameError: name 'frb_pcpi' is not defined

In [33]:
# save frb_pcpi to CSV
frb_pcpi.to_csv('frb_pcpi.csv', index = True)

In [36]:
# Can use list of good ids to download other county datasets from FRED

# Don't download the PCPI data again; it's annual so doesn't update frequently
    # And takes forever to download

# But do unemployment next

# County-level data

In [18]:
import pandas as pd
countyIncome = pd.read_csv('frb_pcpi.csv')

# Reshape countyIncome to long format
countyIncome_long = countyIncome.melt(id_vars=['Unnamed: 0'], var_name='GEOID', value_name='PCPI')

# Rename the 'Unnamed: 0' column to 'Date' for better readability
countyIncome_long.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)

# Change to date-time format
countyIncome_long['Date'] = pd.to_datetime(countyIncome_long['Date'])

# Merge the two datasets on the 'GEOID' column
merged_data = pd.merge(countyIncome_long, FedCounties2, on='GEOID', how='left')

# Group by 'Date' and 'District' and calculate the average PCPI
average_pcpi_per_district = merged_data.groupby(['Date', 'District'])['PCPI'].mean().reset_index()

# Display the first few rows of the result
print(average_pcpi_per_district.head())

# Save the result to a new CSV file
#average_pcpi_per_district.to_csv('path/to/average_pcpi_per_district.csv', index=False)


        Date  District         PCPI
0 1969-01-01         1  3703.651515
1 1969-01-01         2  3925.093333
2 1969-01-01         3  3615.633333
3 1969-01-01         4  3054.674556
4 1969-01-01         5  2842.814570


In [19]:
# Get federal poverty guidelines for family of four
# Source: https://aspe.hhs.gov/topics/poverty-economic-mobility/poverty-guidelines/prior-hhs-poverty-guidelines-federal-register-references
poverty = pd.read_csv('poverty.csv')

# change to date-time format for year only
poverty['Date'] = pd.to_datetime(poverty['Date'], format='%Y')

# Join poverty and merged_data on 'Date'
poverty_merged = pd.merge(poverty, merged_data, on='Date', how='left')

# Calculate the percentage of the federal poverty level
poverty_merged['FPL'] = poverty_merged['PCPI'] / poverty_merged['Poverty']

KeyError: 'Poverty'

In [93]:
poverty_merged.tail()

,Date,poverty_threshold,GEOID,PCPI,FID,STATEFP,COUNTYFP,COUNTYNS,NAME,NAMELSAD,...,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,GEOID_1,District,PovertyID,Below_poverty
166751,2022-01-01,27750,PCPI56041,44775.0,120.0,56.0,41.0,1605084.0,Uinta,Uinta County,...,,A,5.391632e+09,16625817.0,41.284726,-110.558947,56041.0,10.0,S1701ACS056041,False
166752,2022-01-01,27750,PCPI56043,55288.0,600.0,56.0,43.0,1605085.0,Washakie,Washakie County,...,,A,5.798139e+09,10429604.0,43.878830,-107.669052,56043.0,10.0,S1701ACS056043,False
166753,2022-01-01,27750,PCPI56045,50987.0,2120.0,56.0,45.0,1605086.0,Weston,Weston County,...,,A,6.210804e+09,5225499.0,43.846213,-104.570020,56045.0,10.0,S1701ACS056045,False
166754,2023-01-01,30000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
166755,2024-01-01,31200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [92]:
# Count proportion of counties in each district that are below the poverty line
poverty_merged['Below_poverty'] = poverty_merged['PCPI'] < poverty_merged['poverty_threshold']

# Group by 'Date' and 'District' and calculate the proportion of counties below the poverty line
poverty_proportion = poverty_merged.groupby(['Date', 'District'])['Below_poverty'].mean().reset_index()

# Display the first few rows of the result
print(poverty_proportion.head())

        Date  District  Below_poverty
0 1969-01-01       1.0       0.439394
1 1969-01-01       2.0       0.426667
2 1969-01-01       3.0       0.500000
3 1969-01-01       4.0       0.739645
4 1969-01-01       5.0       0.864238


In [43]:
FedCounties2.head()

# Create a new column called PovertyID that we will fill in later
FedCounties2['PovertyID'] = 'S1701ACS0' + FedCounties2['GEOID'].str[-5:]

# Setting up FRED access
import pandas as pd
from fredapi import Fred
api_key = "b40f40a14d67c7903b44700db6b7b6e7"
fred = Fred(api_key=api_key)

C:\Users\MCOB PHD 14\AppData\Local\Temp\ipykernel_13992\2454260172.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FedCounties2['PovertyID'] = 'S1701ACS0' + FedCounties2['GEOID'].str[-5:]


In [44]:
frb_pov = pd.DataFrame()

for series_id in FedCounties2['PovertyID']:
    try:
        frb_data_pov = fred.get_series(series_id)
        frb_pov = pd.concat([frb_pov, frb_data_pov], axis=1)
    except Exception as e:
        print(f"Error occurred while processing series ID {series_id}: {e}")
        continue

Error occurred while processing series ID S1701ACS006093: Too Many Requests.  Exceeded Rate Limit
Error occurred while processing series ID S1701ACS006095: Too Many Requests.  Exceeded Rate Limit
Error occurred while processing series ID S1701ACS006097: Too Many Requests.  Exceeded Rate Limit
Error occurred while processing series ID S1701ACS006099: Too Many Requests.  Exceeded Rate Limit
Error occurred while processing series ID S1701ACS006101: Too Many Requests.  Exceeded Rate Limit
Error occurred while processing series ID S1701ACS006103: Too Many Requests.  Exceeded Rate Limit
Error occurred while processing series ID S1701ACS012079: Too Many Requests.  Exceeded Rate Limit
Error occurred while processing series ID S1701ACS012081: Too Many Requests.  Exceeded Rate Limit
Error occurred while processing series ID S1701ACS012083: Too Many Requests.  Exceeded Rate Limit
Error occurred while processing series ID S1701ACS012085: Too Many Requests.  Exceeded Rate Limit
Error occurred while

In [51]:
countyIncome_long  

,Date,GEOID,PCPI
0,1969-01-01 00:00:00,PCPI01001,2780.0
1,1970-01-01 00:00:00,PCPI01001,3158.0
2,1971-01-01 00:00:00,PCPI01001,3454.0
3,1972-01-01 00:00:00,PCPI01001,3687.0
4,1973-01-01 00:00:00,PCPI01001,4039.0
...,...,...,...
166747,2018-01-01 00:00:00,PCPI56045,44657.0
166748,2019-01-01 00:00:00,PCPI56045,45546.0
166749,2020-01-01 00:00:00,PCPI56045,49854.0
166750,2021-01-01 00:00:00,PCPI56045,49734.0
